In [1]:
import pandas as pd
import numpy as np
df_m = pd.read_csv('Data/MarvelComicCaracters.csv', sep=';')

# Описание данных

- name — оригинальное имя персонажа вселенной Marvel
- align — принадлежность персонажа к доброй/злой/нейтральной стороне
- eye — описание глаз персонажа
- hair — описание волос персонажа
- sex — пол персонажа
- appearances — количество появлений персонажа
- first_appearance — год первого появления персонажа

In [2]:
df_m.head(n=3)

,name,align,eye,hair,sex,appearances,first_appearance
0,Spider-Man (Peter Parker),Добрый персонаж,Карие глаза,Каштановые волосы,Мужской персонаж,4043.0,1962.0
1,Captain America (Steven Rogers),Добрый персонаж,Голубые глаза,Белые волосы,Мужской персонаж,3360.0,1941.0
2,"Wolverine (James \""Logan\"" Howlett)",Нейтральный персонаж,Голубые глаза,Черные волосы,Мужской персонаж,3061.0,1974.0


# Задания

### Сколько пропущенных значений в переменной `appearances`? Выберите один верный ответ.

In [3]:
df_m['appearances'].isna().sum()

1096

### Укажите, верно ли следующее утверждение: наибольший межквартильный размах по переменной `appearances` будет для злых персонажей. Вам понадобятся данные по типу персонажа (переменная `align`), удалите пропущенные значения по столбцу `appearances` и посчитайте требуемую меру разброса.

In [6]:
def get_iqr(a):
    q1 = np.percentile(a, 25)
    q3 = np.percentile(a, 75) 
    return q3 - q1


def get_iqr_outlier_thresholds(df, col, n_iqr):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    
    iqr_outlier_threshold_bottom = q1 - n_iqr * iqr
    iqr_outlier_threshold_up = q3 + n_iqr * iqr
    return iqr_outlier_threshold_bottom, iqr_outlier_threshold_up

In [4]:
df_m \
    .loc[df_m['appearances'].notna(), :] \
    .groupby('align')['appearances'].agg(get_iqr) \
    .sort_values(ascending=False)

align
Добрый персонаж         13.0
Нейтральный персонаж     8.0
Злой персонаж            5.0
Name: appearances, dtype: float64

### Сколько выбросов встречается по переменной `appearances`? Работайте с исходными данными, предварительно удалите пропущенные значения по столбцу `appearances`, используйте для нахождения выбросов межквартильный размах. Выберите один верный ответ.

In [16]:
d.loc[d['appearances'] > get_iqr_outlier_thresholds(d, 'appearances', 1.5)[1]].shape[0]

1938

### На основе данных, полученных в третьем номере, укажите, верно ли следующее утверждение: наибольшее количество выбросов по переменной `appearances` наблюдается у добрых персонажей.

In [39]:
d \
    .loc[d['appearances'] > get_iqr_outlier_thresholds(d, 'appearances', 1.5)[1]] \
    .pivot_table(values='name', index='align', aggfunc='count', dropna=False) \
    .sort_values('name', ascending=False) \
    .index[0]

'Добрый персонаж'

### Создайте новую переменную, которая будет представлять собой прологарифмированную переменную `appearances`. Сколько выбросов получится по новой переменной? Работайте с исходными данными, предварительно удалите пропущенные значения по новому столбцу, используйте для нахождения выбросов три среднеквадратичных отклонения от среднего. Введите ответ в виде целого числа.